# Phase 1b: Data Ingestion & Schema Harmonization

## Objective

Transform raw, messy text and external API data into two clean, separate datasets:

1. **`daily_news_cleaned.csv`**: A harmonized list of unique news events with standardized timestamps
2. **`stock_returns_60.csv`**: A clean record of log-returns for the 60 target stocks, preserving sector and beta metadata

## Data Quality Challenges

Each news source has distinct irregularities that must be handled individually:

| Source | Issues | Solutions |
|--------|--------|----------|
| **CNBC** | Empty rows, "ET" timezone noise, verbose date format | Drop NaN rows, strip timezone, parse datetime |
| **Guardian** | No description column, `\n\n` in text, DMY format | Fill empty descriptions, clean newlines, parse DMY |
| **Reuters** | Clean format (minimal issues) | Direct date parsing (MMM DD YYYY) |

---

## 1. Setup: Imports and Path Configuration

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf

import os
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
project_root = '/content/drive/MyDrive/market-sentiment-impact-analysis'

data_raw = os.path.join(project_root, 'data', 'raw')
data_processed = os.path.join(project_root, 'data', 'processed')
data_tickers = os.path.join(project_root, 'data', 'tickers')

os.makedirs(data_raw, exist_ok=True)
os.makedirs(data_processed, exist_ok=True)
os.makedirs(data_tickers, exist_ok=True)

---

## 2. News Ingestion & Specific Schema Mapping

Each news source requires individual processing due to distinct data quality issues.


### 2.1 CNBC Data Processing

**Input Schema**: `Headlines`, `Time`, `Description`

**Data Irregularities**:
- Empty rows (`,,` patterns)
- Timezone noise: `"7:51 PM ET Fri, 17 July 2020"` contains "ET" and day names
- Verbose date format requiring custom parsing

**Action Plan**:
1. Load CSV, drop fully empty rows
2. Strip " ET" from time strings
3. Parse datetime with custom logic
4. Rename columns to standard schema
5. Add source tag

In [4]:
cnbc_path = os.path.join(data_raw, 'cnbc_headlines.csv')

print(f'Loading CNBC data from {cnbc_path}')
cnbc_raw = pd.read_csv(cnbc_path)

print(f'Initial shape: {cnbc_raw.shape}')
print('\nFirst 5 rows:')
print(cnbc_raw.head())
print(f'\nColumn names: {list(cnbc_raw.columns)}')

Loading CNBC data from /content/drive/MyDrive/market-sentiment-impact-analysis/data/raw/cnbc_headlines.csv
Initial shape: (3080, 3)

First 5 rows:
                                           Headlines                            Time                                        Description
0  Jim Cramer: A better way to invest in the Covi...   7:51  PM ET Fri, 17 July 2020  "Mad Money" host Jim Cramer recommended buying...
1     Cramer's lightning round: I would own Teradyne   7:33  PM ET Fri, 17 July 2020  "Mad Money" host Jim Cramer rings the lightnin...
2                                                NaN                             NaN                                                NaN
3  Cramer's week ahead: Big week for earnings, ev...   7:25  PM ET Fri, 17 July 2020  "We'll pay more for the earnings of the non-Co...
4  IQ Capital CEO Keith Bliss says tech and healt...   4:24  PM ET Fri, 17 July 2020  Keith Bliss, IQ Capital CEO, joins "Closing Be...

Column names: ['Headlines', 'Time', 

In [5]:
# Drop completely empty rows
cnbc_clean = cnbc_raw.dropna(how='all')

rows_removed = len(cnbc_raw) - len(cnbc_clean)
print(f'Removed {rows_removed} completely empty rows')
print(f'Shape after cleaning: {cnbc_clean.shape}')

Removed 280 completely empty rows
Shape after cleaning: (2800, 3)


In [6]:
# Clean the Time colum, remove " ET" timezone marker
# Example: "7:51  PM ET Fri, 17 July 2020" -> "7:51  PM Fri, 17 July 2020"

def clean_cnbc_time(time_str):
    if pd.isna(time_str):
        return time_str
    cleaned = str(time_str).replace(' ET', '')
    return cleaned

cnbc_clean['Time_Cleaned'] = cnbc_clean['Time'].apply(clean_cnbc_time)

print('Sample of cleaned timestamps:')
print(cnbc_clean[['Time', 'Time_Cleaned']].head(3))

Sample of cleaned timestamps:
                             Time                 Time_Cleaned
0   7:51  PM ET Fri, 17 July 2020   7:51  PM Fri, 17 July 2020
1   7:33  PM ET Fri, 17 July 2020   7:33  PM Fri, 17 July 2020
3   7:25  PM ET Fri, 17 July 2020   7:25  PM Fri, 17 July 2020


In [7]:
# Parse datetime

def parse_cnbc_datetime(time_str):
    if pd.isna(time_str):
        return pd.NaT
    try:
        return pd.to_datetime(time_str, format='mixed', utc=False)
    except:
        return pd.NaT

cnbc_clean['date'] = cnbc_clean['Time_Cleaned'].apply(parse_cnbc_datetime)

print('Date parsing results:')
print(f'Successfully parsed: {cnbc_clean['date'].notna().sum()}')
print(f'Failed to parse: {cnbc_clean['date'].isna().sum()}')
print('\nSample parsed dates:')
print(cnbc_clean[['Time_Cleaned', 'date']].head())

Date parsing results:
Successfully parsed: 2800
Failed to parse: 0

Sample parsed dates:
                  Time_Cleaned                date
0   7:51  PM Fri, 17 July 2020 2020-07-17 19:51:00
1   7:33  PM Fri, 17 July 2020 2020-07-17 19:33:00
3   7:25  PM Fri, 17 July 2020 2020-07-17 19:25:00
4   4:24  PM Fri, 17 July 2020 2020-07-17 16:24:00
5   7:36  PM Thu, 16 July 2020 2020-07-16 19:36:00


In [8]:
# Schema standardization
# Time -> date, Headlines -> headline, Description -> description
# Add: source = "CNBC"

cnbc_standard = cnbc_clean[['date', 'Headlines', 'Description']].copy()
cnbc_standard.columns = ['date', 'headline', 'description']
cnbc_standard['source'] = 'CNBC'

cnbc_standard = cnbc_standard.dropna(subset=['date'])

print(f'CNBC standardized shape: {cnbc_standard.shape}')
print(f'\nColumns: {list(cnbc_standard.columns)}')
print('\nSample:')
print(cnbc_standard.head(3))

CNBC standardized shape: (2800, 4)

Columns: ['date', 'headline', 'description', 'source']

Sample:
                 date                                           headline                                        description source
0 2020-07-17 19:51:00  Jim Cramer: A better way to invest in the Covi...  "Mad Money" host Jim Cramer recommended buying...   CNBC
1 2020-07-17 19:33:00     Cramer's lightning round: I would own Teradyne  "Mad Money" host Jim Cramer rings the lightnin...   CNBC
3 2020-07-17 19:25:00  Cramer's week ahead: Big week for earnings, ev...  "We'll pay more for the earnings of the non-Co...   CNBC


### 2.2 Guardian Data Processing

**Input Schema**: `Time`, `Headlines` (Note: Reversed order, no Description)

**Data Irregularities**:
- Missing `description` column entirely
- Dirty text: `\n\n` patterns in headlines
- Date format: `18-Jul-20` (Day-Month-Year)

**Action Plan**:
1. Load CSV
2. Clean newlines from Headlines
3. Parse DMY date format
4. Create empty `description` column (use `""`, not NaN)
5. Rename and add source tag

In [9]:
guardian_path = os.path.join(data_raw, 'guardian_headlines.csv')

print(f'Loading Guardian data from {guardian_path}')
guardian_raw = pd.read_csv(guardian_path)

print(f'Initial shape: {guardian_raw.shape}')
print(f'First 10 rows:')
print(guardian_raw.head(10))
print(f'Column names: {list(guardian_raw.columns)}')

Loading Guardian data from /content/drive/MyDrive/market-sentiment-impact-analysis/data/raw/guardian_headlines.csv
Initial shape: (17800, 2)
First 10 rows:
        Time                                          Headlines
0  18-Jul-20   Johnson is asking Santa for a Christmas recovery
1  18-Jul-20  ‘I now fear the worst’: four grim tales of wor...
2  18-Jul-20  Five key areas Sunak must tackle to serve up e...
3  18-Jul-20  Covid-19 leaves firms ‘fatally ill-prepared’ f...
4  18-Jul-20  The Week in Patriarchy  \n\n\n  Bacardi's 'lad...
5  18-Jul-20  English councils call for smoking ban outside ...
6  18-Jul-20              Can Tesla justify a $300bn valuation?
7  18-Jul-20  Empty city centres: 'I’m not sure it will ever...
8  18-Jul-20  Democratising finance for all? An investment a...
9  18-Jul-20  Homebuyer loses £300,000 to fraudsters – but g...
Column names: ['Time', 'Headlines']


In [10]:
# Clean newlines from Headlines

def clean_newlines(text):
    """Replace multiple newlines and tabs with single space."""
    if pd.isna(text):
        return text

    cleaned = str(text).replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    cleaned = ' '.join(cleaned.split())
    return cleaned

guardian_raw['Headlines_Cleaned'] = guardian_raw['Headlines'].apply(clean_newlines)

has_newlines = guardian_raw['Headlines'].str.contains('\n', na=False)
print(f'Headlines with newlines: {has_newlines.sum()}')
print(f'\nBefore/After sample:')
if has_newlines.sum() > 0:
    sample_idx = guardian_raw[has_newlines].index[0]
    print(f'BEFORE: {repr(guardian_raw.loc[sample_idx, 'Headlines'])}')
    print(f'AFTER:  {guardian_raw.loc[sample_idx, 'Headlines_Cleaned']}')

Headlines with newlines: 1246

Before/After sample:
BEFORE: "The Week in Patriarchy  \n\n\n  Bacardi's 'lady vodka': the latest in a long line of depressing gendered products"
AFTER:  The Week in Patriarchy Bacardi's 'lady vodka': the latest in a long line of depressing gendered products


In [11]:
from pandas.core.dtypes.missing import isna
# Parse date - format is "18-Jul-20" (DMY)

def parse_guardian_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        return pd.to_datetime(date_str, format='%d-%b-%y')
    except:
        return pd.NaT

guardian_raw['date'] = guardian_raw['Time'].apply(parse_guardian_date)

print('Date parsing results:')
print(f'Successfully parsed: {guardian_raw['date'].notna().sum()}')
print(f'Failed to parse: {guardian_raw['date'].isna().sum()}')
print('\nDate range:')
print(f'Earliest: {guardian_raw['date'].min()}')
print(f'Latest: {guardian_raw['date'].max()}')

Date parsing results:
Successfully parsed: 17760
Failed to parse: 40

Date range:
Earliest: 2017-12-17 00:00:00
Latest: 2020-07-18 00:00:00


In [12]:
# Schema standardization

guardian_standard = pd.DataFrame({
    'date': guardian_raw['date'],
    'headline': guardian_raw['Headlines_Cleaned'],
    'description': '',
    'source': 'Guardian'
})

guardian_standard = guardian_standard.dropna(subset=['date'])

print(f'Guardian standardized shape: {guardian_standard.shape}')
print(f'\nColumns: {list(guardian_standard.columns)}')
print('\nDescription column check:')
print(f'  Type: {type(guardian_standard['description'].iloc[0])}')
print(f'  Is NaN: {guardian_standard['description'].isna().sum()}')
print(f'  Is empty string: {(guardian_standard['description'] == '').sum()}')
print('\nSample:')
print(guardian_standard.head(3))

Guardian standardized shape: (17760, 4)

Columns: ['date', 'headline', 'description', 'source']

Description column check:
  Type: <class 'str'>
  Is NaN: 0
  Is empty string: 17760

Sample:
        date                                           headline description    source
0 2020-07-18   Johnson is asking Santa for a Christmas recovery              Guardian
1 2020-07-18  ‘I now fear the worst’: four grim tales of wor...              Guardian
2 2020-07-18  Five key areas Sunak must tackle to serve up e...              Guardian


### 2.3 Reuters Data Processing

**Input Schema**: `Headlines`, `Time`, `Description`

**Data Irregularities**: Minimal - clean format `"Jul 18 2020"`

**Action Plan**:
1. Load CSV
2. Parse clean date format (MMM DD YYYY)
3. Rename columns to standard schema
4. Add source tag

In [13]:
reuters_path = os.path.join(data_raw, 'reuters_headlines.csv')

print(f'Loading Reuters data from: {reuters_path}')
reuters_raw = pd.read_csv(reuters_path)

print(f'Initial shape: {reuters_raw.shape}')
print('\nFirst 5 rows:')
print(reuters_raw.head())
print(f'\nColumn names: {list(reuters_raw.columns)}')

Loading Reuters data from: /content/drive/MyDrive/market-sentiment-impact-analysis/data/raw/reuters_headlines.csv
Initial shape: (32770, 3)

First 5 rows:
                                           Headlines         Time                                        Description
0  TikTok considers London and other locations fo...  Jul 18 2020  TikTok has been in discussions with the UK gov...
1  Disney cuts ad spending on Facebook amid growi...  Jul 18 2020  Walt Disney  has become the latest company to ...
2  Trail of missing Wirecard executive leads to B...  Jul 18 2020  Former Wirecard  chief operating officer Jan M...
3  Twitter says attackers downloaded data from up...  Jul 18 2020  Twitter Inc said on Saturday that hackers were...
4  U.S. Republicans seek liability protections as...  Jul 17 2020  A battle in the U.S. Congress over a new coron...

Column names: ['Headlines', 'Time', 'Description']


In [14]:
# Parse date, format is "Jul 18 2020" (MMM DD YYYY)

def parse_reuters_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        return pd.to_datetime(date_str, format='%b %d %Y')
    except:
        return pd.NaT

reuters_raw['date'] = reuters_raw['Time'].apply(parse_reuters_date)

print('Date parsing results:')
print(f'Successfully parsed: {reuters_raw['date'].notna().sum()}')
print(f'Failed to parse: {reuters_raw['date'].isna().sum()}')
print('\nDate range:')
print(f'Earliest: {reuters_raw['date'].min()}')
print(f'Latest: {reuters_raw['date'].max()}')
print('\nSample parsed dates:')
print(reuters_raw[['Time', 'date']].head())

Date parsing results:
Successfully parsed: 32770
Failed to parse: 0

Date range:
Earliest: 2018-03-20 00:00:00
Latest: 2020-07-18 00:00:00

Sample parsed dates:
          Time       date
0  Jul 18 2020 2020-07-18
1  Jul 18 2020 2020-07-18
2  Jul 18 2020 2020-07-18
3  Jul 18 2020 2020-07-18
4  Jul 17 2020 2020-07-17


In [15]:
# Schema standardization

reuters_standard = pd.DataFrame({
    'date': reuters_raw['date'],
    'headline': reuters_raw['Headlines'],
    'description': reuters_raw['Description'],
    'source': 'Reuters'
})

reuters_standard = reuters_standard.dropna(subset=['date'])

print(f'Reuters standardized shape: {reuters_standard.shape}')
print(f'Columns: {list(reuters_standard.columns)}')
print('\nSample:')
print(reuters_standard.head(3))

Reuters standardized shape: (32770, 4)
Columns: ['date', 'headline', 'description', 'source']

Sample:
        date                                           headline                                        description   source
0 2020-07-18  TikTok considers London and other locations fo...  TikTok has been in discussions with the UK gov...  Reuters
1 2020-07-18  Disney cuts ad spending on Facebook amid growi...  Walt Disney  has become the latest company to ...  Reuters
2 2020-07-18  Trail of missing Wirecard executive leads to B...  Former Wirecard  chief operating officer Jan M...  Reuters


---

## 3. Global News Standardization

Now that all sources have identical schemas (`date`, `headline`, `description`, `source`), we merge and refine.

### 3.1 Concatenation & Time Filtering

In [16]:
from numpy._core.defchararray import index
# Combine all three news sources

news_sources = [cnbc_standard, guardian_standard, reuters_standard]

news_combined = pd.concat(news_sources, ignore_index=True)

print(f'Combined news shape: {news_combined.shape}')
print('\nRecords per source:')
print(news_combined['source'].value_counts())
print('\nDate range before filtering:')
print(f'  Earliest: {news_combined['date'].min()}')
print(f'  Latest: {news_combined['date'].max()}')

Combined news shape: (53330, 4)

Records per source:
source
Reuters     32770
Guardian    17760
CNBC         2800
Name: count, dtype: int64

Date range before filtering:
  Earliest: 2017-12-17 00:00:00
  Latest: 2020-07-18 00:00:00


In [17]:
# Target range: 2018-01-01 to 2020-07-19

news_combined['date'] = pd.to_datetime(news_combined['date'], utc=True)

start_date = pd.Timestamp('2018-01-01', tz='UTC')
end_date = pd.Timestamp('2020-07-19', tz='UTC')

print('Applying hard cut filter:')
print(f'  Start: {start_date.date()}')
print(f'  End: {end_date.date()}')

news_filtered = news_combined[
    (news_combined['date'] >= start_date) &
    (news_combined['date'] <= end_date)
].copy()

news_filtered = news_filtered.sort_values('date').reset_index(drop=True)

rows_removed = len(news_combined) - len(news_filtered)
print(f'\nRows removed (outside date range): {rows_removed}')
print(f'Rows retained: {len(news_filtered)}')
print('\nFinal date range:')
print(f'  Earliest: {news_filtered['date'].min().date()}')
print(f'  Latest: {news_filtered['date'].max().date()}')

Applying hard cut filter:
  Start: 2018-01-01
  End: 2020-07-19

Rows removed (outside date range): 245
Rows retained: 53085

Final date range:
  Earliest: 2018-01-02
  Latest: 2020-07-18


### 3.2 Text Engineering: Creating the "Context" Field

**Problem**: Headlines alone are too short for FinBERT b/c transformers work best with full sentences

**Solution**: Concatenate `headline + ". " + description`

**Edge Cases**:
- Guardian has empty description → result should be just headline
- No trailing ". " for empty descriptions

In [18]:
# Create final_text by concatenating headline and description

def create_final_text(row):
    headline = str(row['headline']) if pd.notna(row['headline']) else ''
    description = str(row['description']) if pd.notna(row['description']) else ''

    if description.strip() == '':
        return headline.strip()
    else:
        return f"{headline.strip()}. {description.strip()}"

news_filtered['final_text'] = news_filtered.apply(create_final_text, axis=1)
print('Text engineering results:')
print(f'\nSample from CNBC (has description):')
cnbc_sample = news_filtered[news_filtered['source'] == 'CNBC'].iloc[0]
print(f'Headline: {cnbc_sample['headline'][:100]}...')
print(f'Description: {cnbc_sample['description'][:100]}...')
print(f'Final Text: {cnbc_sample['final_text'][:150]}...')

print('\nSample from Guardian (no description):')
guardian_sample = news_filtered[news_filtered['source'] == 'Guardian'].iloc[0]
print(f'Headline: {guardian_sample['headline'][:100]}...')
print(f"Description: '{guardian_sample['description']}'")
print(f'Final Text: {guardian_sample['final_text'][:150]}...')

Text engineering results:

Sample from CNBC (has description):
Headline: The major indexes trade lower in January...
Description: Our data partners at Kensho say beware of January. Since 2012 on average the Nasdaq, S&P 500, Dow an...
Final Text: The major indexes trade lower in January. Our data partners at Kensho say beware of January. Since 2012 on average the Nasdaq, S&P 500, Dow and Russel...

Sample from Guardian (no description):
Headline: Former advertising executive reveals junk food-pushing tactics...
Description: ''
Final Text: Former advertising executive reveals junk food-pushing tactics...


In [19]:
nan_count = news_filtered['final_text'].isna().sum()
empty_count = (news_filtered['final_text'].str.strip() == '').sum()

print('Validation Results:')
print(f'  NaN values in final_text: {nan_count}')
print(f'  Empty strings in final_text: {empty_count}')

news_filtered['text_length'] = news_filtered['final_text'].str.len()
print(f'\nText length statistics:')
print(news_filtered['text_length'].describe())

Validation Results:
  NaN values in final_text: 0
  Empty strings in final_text: 0

Text length statistics:
count    53085.000000
mean       206.593444
std        105.747685
min         18.000000
25%         74.000000
50%        248.000000
75%        295.000000
max        566.000000
Name: text_length, dtype: float64


### 3.3 Deduplication: Bias Removal

**Problem**: Wire services (Reuters/CNBC/Guardian) often syndicate the same story

**Impact**: Duplicate stories artificially inflate sentiment on certain dates

**Solution**: Drop duplicates based on `final_text`, keeping first occurrence

In [20]:
total_before = len(news_filtered)
duplicates = news_filtered['final_text'].duplicated(keep=False).sum()

print('Duplication Analysis:')
print(f'  Total records: {total_before}')
print(f'  Duplicate texts (all occurrences): {duplicates}')
print(f'  Unique texts: {news_filtered['final_text'].nunique()}')

if duplicates > 0:
    dup_example = news_filtered[news_filtered['final_text'].duplicated(keep=False)].head(4)
    print(f'\nSample duplicate entries:')
    print(dup_example[['date', 'source', 'final_text']])

Duplication Analysis:
  Total records: 53085
  Duplicate texts (all occurrences): 212
  Unique texts: 52974

Sample duplicate entries:
                         date source                                         final_text
73  2018-01-04 19:06:00+00:00   CNBC  Cramer Remix: Even at Dow 25K, this market is ...
74  2018-01-04 19:21:00+00:00   CNBC  Cramer Remix: Even at Dow 25K, this market is ...
102 2018-01-05 17:18:00+00:00   CNBC  Cramer: The S&P 500's winners for 2017 could s...
103 2018-01-05 18:54:00+00:00   CNBC  Cramer says the S&P 500's biggest 2017 losers ...


In [21]:
news_deduped = news_filtered.drop_duplicates(subset=['final_text'], keep='first')

total_after = len(news_deduped)
removed = total_before - total_after
removal_pct = (removed / total_before) * 100

print('\nDeduplication Results:')
print(f'  Before: {total_before:,} records')
print(f'  After: {total_after:,} records')
print(f'  Removed: {removed:,} duplicates ({removal_pct:.2f}%)')


Deduplication Results:
  Before: 53,085 records
  After: 52,974 records
  Removed: 111 duplicates (0.21%)


### 3.4 Final News Dataset Preparation

In [22]:
# Select final columns for output
# Keep: date, source, final_text

news_final = news_deduped[['date', 'source', 'final_text']].copy()
news_final = news_final.reset_index(drop=True)

print('Final news dataset:')
print(f'  Shape: {news_final.shape}')
print(f'  Columns: {list(news_final.columns)}')
print(f'  Date range: {news_final['date'].min().date()} to {news_final['date'].max().date()}')
print('  Records per source:')
print(news_final['source'].value_counts())
print('\nFirst 3 records:')
print(news_final.head(3))

Final news dataset:
  Shape: (52974, 3)
  Columns: ['date', 'source', 'final_text']
  Date range: 2018-01-02 to 2020-07-18
  Records per source:
source
Reuters     32673
Guardian    17516
CNBC         2785
Name: count, dtype: int64

First 3 records:
                       date    source                                         final_text
0 2018-01-02 00:00:00+00:00  Guardian  Former advertising executive reveals junk food...
1 2018-01-02 00:00:00+00:00  Guardian  Transport secretary ‘running scared’ as he fli...
2 2018-01-02 00:00:00+00:00  Guardian  Good for factories, bad for shoppers: a Brexit...


## 4. Stock Data Integration

Preparing the target variables for statistical analysis.

### 4.1 Ticker Aggregation & Metadata Loading

In [23]:
high_beta_path = os.path.join(data_tickers, 'high_beta.csv')
low_beta_path = os.path.join(data_tickers, 'low_beta.csv')

high_beta_df = pd.read_csv(high_beta_path)
low_beta_df = pd.read_csv(low_beta_path)

print(f'\nHigh Beta stocks: {len(high_beta_df)}')
print(high_beta_df[['Ticker', 'Security', 'GICS Sector', 'Beta']].head())

print(f"\nLow Beta stocks: {len(low_beta_df)}")
print(low_beta_df[['Ticker', 'Security', 'GICS Sector', 'Beta']].head())


High Beta stocks: 30
  Ticker                        Security             GICS Sector      Beta
0    APA                 APA Corporation                  Energy  1.950480
1   NCLH  Norwegian Cruise Line Holdings  Consumer Discretionary  1.936777
2   CVNA                         Carvana  Consumer Discretionary  1.919910
3    CCL                        Carnival  Consumer Discretionary  1.837347
4   TRGP                 Targa Resources                  Energy  1.830136

Low Beta stocks: 30
  Ticker                    Security       GICS Sector      Beta
0    NEM                     Newmont         Materials  0.290109
1    CLX                      Clorox  Consumer Staples  0.330349
2     KR                      Kroger  Consumer Staples  0.342843
3    CPB    Campbell's Company (The)  Consumer Staples  0.345294
4    SJM  J.M. Smucker Company (The)  Consumer Staples  0.385139


In [24]:
high_beta_df['Beta_Group'] = 'High Beta'
low_beta_df['Beta_Group'] = 'Low Beta'

ticker_metadata = pd.concat([high_beta_df, low_beta_df], ignore_index=True)

all_tickers = ticker_metadata['Ticker'].tolist()

print(f'\nTotal tickers: {len(all_tickers)}')
print(f'High Beta: {len(high_beta_df)}')
print(f'Low Beta: {len(low_beta_df)}')
print(f'\nTicker list: {all_tickers}')


Total tickers: 60
High Beta: 30
Low Beta: 30

Ticker list: ['APA', 'NCLH', 'CVNA', 'CCL', 'TRGP', 'RCL', 'MGM', 'HAL', 'AMP', 'OXY', 'XYZ', 'DVN', 'ON', 'TPR', 'BA', 'FCX', 'URI', 'MPC', 'WDC', 'PRU', 'LRCX', 'SYF', 'WYNN', 'OKE', 'KEY', 'C', 'IVZ', 'COF', 'FANG', 'TTD', 'NEM', 'CLX', 'KR', 'CPB', 'SJM', 'GIS', 'HRL', 'SW', 'CAG', 'CHD', 'VZ', 'DPZ', 'WMT', 'KDP', 'ED', 'KMB', 'DG', 'AEP', 'CHRW', 'PSA', 'EXR', 'WEC', 'GILD', 'CMS', 'CL', 'GEN', 'MKC', 'COST', 'PG', 'LNT']


In [25]:
ticker_map = ticker_metadata.set_index('Ticker')[['GICS Sector', 'Beta', 'Beta_Group']].to_dict('index')

print('\nSample ticker mapping:')
sample_ticker = all_tickers[0]
print(f'{sample_ticker}: {ticker_map[sample_ticker]}')


Sample ticker mapping:
APA: {'GICS Sector': 'Energy', 'Beta': 1.9504802306078584, 'Beta_Group': 'High Beta'}


### 4.2 Batch Price Fetching

In [26]:
# Download historical prices for all 60 tickers: 2018-01-01 to 2020-07-19

end_date = '2020-07-19'
start_date = '2018-01-01'

print('\nDownloading stock data...')
print(f'  Tickers: {len(all_tickers)}')
print(f'  Date range: {start_date} to {end_date}')
print(f'  Interval: Daily')

prices_raw = yf.download(
    all_tickers,
    start=start_date,
    end=end_date,
    interval='1d',
    auto_adjust=True,
    progress=False
)['Close']

print(f'Shape: {prices_raw.shape} (rows=trading days, columns=stocks)')
print(f'Date range: {prices_raw.index[0]} to {prices_raw.index[-1]}')
print(f'Trading days: {len(prices_raw)}')


  Tickers: 60
  Date range: 2018-01-01 to 2020-07-19
  Interval: Daily
Shape: (640, 60) (rows=trading days, columns=stocks)
Date range: 2018-01-02 00:00:00 to 2020-07-17 00:00:00
Trading days: 640


In [27]:
missing_data = prices_raw.isna().sum()
stocks_with_missing = missing_data[missing_data > 0].sort_values(ascending=False)

print(f'Stocks with missing data: {len(stocks_with_missing)}')

if len(stocks_with_missing) > 0:
    print(f'\nStocks with most missing data:')
    print(stocks_with_missing.head(10))

    pct_missing = (stocks_with_missing / len(prices_raw) * 100)
    print(f'\nPercentage missing:')
    print(pct_missing.head(10).round(2))
else:
    print('No missing data detected')

Stocks with missing data: 0
No missing data detected


In [28]:
prices_filled = prices_raw.fillna(method='ffill')
prices_clean = prices_filled.dropna(axis=0, how='any')

print('After cleaning:')
print(f'  Original days: {len(prices_raw)}')
print(f'  Clean days: {len(prices_clean)}')
print(f'  Days removed: {len(prices_raw) - len(prices_clean)}')
print('Price data ready for return calculation')

After cleaning:
  Original days: 640
  Clean days: 640
  Days removed: 0
Price data ready for return calculation


### 4.3 Feature Engineering: Log Returns

**Mathematical Foundation**:

$$R_t = \ln\left(\frac{P_t}{P_{t-1}}\right) = \ln(P_t) - \ln(P_{t-1})$$

In [29]:
returns = np.log(prices_clean / prices_clean.shift(1))
returns = returns.dropna()

print('Returns calculated:')
print(f'  Shape: {returns.shape}')
print(f'  Date range: {returns.index[0]} to {returns.index[-1]}')
print('\nSample returns (first stock, first 5 days):')
print(returns.iloc[:5, 0])

Returns calculated:
  Shape: (639, 60)
  Date range: 2018-01-03 00:00:00 to 2020-07-17 00:00:00

Sample returns (first stock, first 5 days):
Date
2018-01-03   -0.008460
2018-01-04   -0.011909
2018-01-05   -0.002116
2018-01-08    0.008719
2018-01-09   -0.011831
Name: AEP, dtype: float64


In [30]:
extreme_returns = (returns.abs() > 0.3).sum().sum()
infinite_returns = np.isinf(returns).sum().sum()

print('Return quality checks:')
print(f'  Extreme returns (|R| > 30%): {extreme_returns}')
print(f'  Infinite values: {infinite_returns}')

Return quality checks:
  Extreme returns (|R| > 30%): 31
  Infinite values: 0


In [31]:
print('\nReturn Statistics (across all stocks):')
print(f'  Mean: {returns.mean().mean():.6f}')
print(f'  Median: {returns.median().median():.6f}')
print(f'  Std Dev: {returns.std().mean():.6f}')
print(f'  Min: {returns.min().min():.6f}')
print(f'  Max: {returns.max().max():.6f}')

annualized_vol = returns.std() * np.sqrt(252)
print(f'\nAnnualized Volatility (mean): {annualized_vol.mean():.3f}')
print(f'  High Beta stocks: {annualized_vol[high_beta_df['Ticker']].mean():.3f}')
print(f'  Low Beta stocks: {annualized_vol[low_beta_df['Ticker']].mean():.3f}')


Return Statistics (across all stocks):
  Mean: -0.000087
  Median: 0.000899
  Std Dev: 0.028545
  Min: -0.773592
  Max: 0.360404

Annualized Volatility (mean): 0.453
  High Beta stocks: 0.616
  Low Beta stocks: 0.291


### 4.4 Reshape to Long Format & Enrich with Metadata

In [32]:
# Convert wide format (date × ticker) to long format (date-ticker pairs)

returns_long = returns.reset_index()

returns_long = returns_long.melt(
    id_vars='Date',
    var_name='Ticker',
    value_name='Log_Return'
)

print('Reshaped to long format:')
print(f'  Shape: {returns_long.shape}')
print(f'  Columns: {list(returns_long.columns)}')
print('\nSample:')
print(returns_long.head(10))

Reshaped to long format:
  Shape: (38340, 3)
  Columns: ['Date', 'Ticker', 'Log_Return']

Sample:
        Date Ticker  Log_Return
0 2018-01-03    AEP   -0.008460
1 2018-01-04    AEP   -0.011909
2 2018-01-05    AEP   -0.002116
3 2018-01-08    AEP    0.008719
4 2018-01-09    AEP   -0.011831
5 2018-01-10    AEP   -0.015420
6 2018-01-11    AEP   -0.011141
7 2018-01-12    AEP   -0.018651
8 2018-01-16    AEP    0.000593
9 2018-01-17    AEP    0.011050


In [33]:
def add_metadata(row):
    ticker = row['Ticker']
    if ticker in ticker_map:
        return pd.Series({
            'Sector': ticker_map[ticker]['GICS Sector'],
            'Beta': ticker_map[ticker]['Beta'],
            'Beta_Group': ticker_map[ticker]['Beta_Group']
        })
    else:
        return pd.Series({
            'Sector': None,
            'Beta': None,
            'Beta_Group': None
        })

metadata_cols = returns_long['Ticker'].apply(lambda t: pd.Series(ticker_map.get(t, {})))
returns_enriched = pd.concat([returns_long, metadata_cols], axis=1)

if 'GICS Sector' in returns_enriched.columns:
    returns_enriched.rename(columns={'GICS Sector': 'Sector'}, inplace=True)

print(f'\nEnriched with metadata:')
print(f'  Shape: {returns_enriched.shape}')
print(f'  Columns: {list(returns_enriched.columns)}')
print('\nSample with metadata:')
print(returns_enriched.head())


Enriched with metadata:
  Shape: (38340, 6)
  Columns: ['Date', 'Ticker', 'Log_Return', 'Sector', 'Beta', 'Beta_Group']

Sample with metadata:
        Date Ticker  Log_Return     Sector      Beta Beta_Group
0 2018-01-03    AEP   -0.008460  Utilities  0.585551   Low Beta
1 2018-01-04    AEP   -0.011909  Utilities  0.585551   Low Beta
2 2018-01-05    AEP   -0.002116  Utilities  0.585551   Low Beta
3 2018-01-08    AEP    0.008719  Utilities  0.585551   Low Beta
4 2018-01-09    AEP   -0.011831  Utilities  0.585551   Low Beta


In [34]:
returns_final = returns_enriched[['Date', 'Ticker', 'Log_Return', 'Sector', 'Beta', 'Beta_Group']].copy()
returns_final = returns_final.sort_values(['Date', 'Ticker']).reset_index(drop=True)

print('Final stock returns dataset:')
print(f'  Shape: {returns_final.shape}')
print(f'  Date range: {returns_final['Date'].min()} to {returns_final['Date'].max()}')
print(f'  Unique tickers: {returns_final['Ticker'].nunique()}')
print(f'  Observations per ticker: {len(returns_final) / returns_final['Ticker'].nunique():.0f}')
print('\nRecords by Beta Group:')
print(returns_final['Beta_Group'].value_counts())

Final stock returns dataset:
  Shape: (38340, 6)
  Date range: 2018-01-03 00:00:00 to 2020-07-17 00:00:00
  Unique tickers: 60
  Observations per ticker: 639

Records by Beta Group:
Beta_Group
Low Beta     19170
High Beta    19170
Name: count, dtype: int64


## 5. Validation & Output

### 5.1 Validation Checks

In [39]:
# News data validation
# Date range

news_min = news_final['date'].min()
news_max = news_final['date'].max()
expected_start = pd.Timestamp('2018-01-01', tz='UTC')
expected_end = pd.Timestamp('2020-07-19', tz='UTC')

date_check = (news_min >= expected_start) and (news_max <= expected_end)
print(f'\n1. Date Range Check:')
print(f'   Expected: {expected_start.date()} to {expected_end.date()}')
print(f'   Actual: {news_min.date()} to {news_max.date()}')
print(f'   Status: {'Pass' if date_check else 'Fail'}')

# No NaN in final_text

nan_text = news_final['final_text'].isna().sum()
print(f'\n2. Text Completeness Check:')
print(f'   NaN values in final_text: {nan_text}')
print(f'   Status: {'Pass' if nan_text == 0 else 'Fail'}')

# No empty strings

empty_text = (news_final['final_text'].str.strip() == '').sum()
print(f'\n3. Empty Text Check:')
print(f'   Empty strings: {empty_text}')
print(f'   Status: {'Pass' if empty_text == 0 else 'Warning'}')

# Unique count (post-deduplication)
total_records = len(news_final)
unique_texts = news_final['final_text'].nunique()
print(f'\n4. Deduplication Check:')
print(f'   Total records: {total_records}')
print(f'   Unique texts: {unique_texts}')
print(f'   Duplicates remaining: {total_records - unique_texts}')
print(f'   Status: {'Pass' if total_records == unique_texts else 'Warning'}')

if all([date_check, nan_text == 0, empty_text == 0, total_records == unique_texts]):
    print('\nAll checks passed')
else:
    print('\nValidation failed')


1. Date Range Check:
   Expected: 2018-01-01 to 2020-07-19
   Actual: 2018-01-02 to 2020-07-18
   Status: Pass

2. Text Completeness Check:
   NaN values in final_text: 0
   Status: Pass

3. Empty Text Check:
   Empty strings: 0
   Status: Pass

4. Deduplication Check:
   Total records: 52974
   Unique texts: 52974
   Duplicates remaining: 0
   Status: Pass

All checks passed


In [43]:
# Stock data validation
# Correct number of tickers

unique_tickers = returns_final['Ticker'].nunique()
expected_tickers = 60
ticker_check = (unique_tickers == expected_tickers)
print(f'\n1. Ticker Count Check:')
print(f'   Expected: {expected_tickers}')
print(f'   Actual: {unique_tickers}')
print(f'   Status: {'Pass' if ticker_check else 'Fail'}')

# No infinite returns
inf_returns = np.isinf(returns_final['Log_Return']).sum()
print(f'\n2. Infinite Returns Check:')
print(f'   Infinite values: {inf_returns}')
print(f'   Status: {'Pass' if inf_returns == 0 else 'Fail'}')

# No NaN returns
nan_returns = returns_final['Log_Return'].isna().sum()
print(f'\n3. NaN Returns Check:')
print(f'   NaN values: {nan_returns}')
print(f'   Status: {'Pass' if nan_returns == 0 else 'Warning'}')

# Metadata completeness
missing_sector = returns_final['Sector'].isna().sum()
missing_beta = returns_final['Beta'].isna().sum()
missing_group = returns_final['Beta_Group'].isna().sum()
metadata_check = (missing_sector == 0) and (missing_beta == 0) and (missing_group == 0)
print(f'\n4. Metadata Completeness Check:')
print(f'   Missing Sector: {missing_sector}')
print(f'   Missing Beta: {missing_beta}')
print(f'   Missing Beta_Group: {missing_group}')
print(f'   Status: {'Pass' if metadata_check else 'Fail'}')

# Beta group distribution
beta_dist = returns_final.groupby('Ticker')['Beta_Group'].first().value_counts()
print(f'\n5. Beta Group Distribution:')
print(beta_dist)
balanced_check = (beta_dist['High Beta'] == 30) and (beta_dist['Low Beta'] == 30)
print(f'   Status: {'Pass' if balanced_check else 'Warning'}')

if all([ticker_check, inf_returns == 0, nan_returns == 0, metadata_check, balanced_check]):
    print('\nAll checks passed')
else:
    print('\nValidation failed')


1. Ticker Count Check:
   Expected: 60
   Actual: 60
   Status: Pass

2. Infinite Returns Check:
   Infinite values: 0
   Status: Pass

3. NaN Returns Check:
   NaN values: 0
   Status: Pass

4. Metadata Completeness Check:
   Missing Sector: 0
   Missing Beta: 0
   Missing Beta_Group: 0
   Status: Pass

5. Beta Group Distribution:
Beta_Group
Low Beta     30
High Beta    30
Name: count, dtype: int64
   Status: Pass

All checks passed


### 5.2 Save Final Datasets

In [44]:
news_output_path = os.path.join(data_processed, 'daily_news_cleaned.csv')
stock_output_path = os.path.join(data_processed, 'stock_returns_60.csv')

print(f'Saving datasets...')
print(f'\nNews data → {news_output_path}')
print(f'Stock data → {stock_output_path}')

Saving datasets...

News data → /content/drive/MyDrive/market-sentiment-impact-analysis/data/processed/daily_news_cleaned.csv
Stock data → /content/drive/MyDrive/market-sentiment-impact-analysis/data/processed/stock_returns_60.csv


In [45]:
news_final.to_csv(news_output_path, index=False)

print(f'\nNews data saved')
print(f'  Records: {len(news_final):,}')
print(f'  Columns: {list(news_final.columns)}')
print(f'  File size: {os.path.getsize(news_output_path) / 1024 / 1024:.2f} MB')


News data saved
  Records: 52,974
  Columns: ['date', 'source', 'final_text']
  File size: 12.29 MB


In [46]:
returns_final.to_csv(stock_output_path, index=False)

print(f'\nStock data saved')
print(f'  Records: {len(returns_final):,}')
print(f'  Columns: {list(returns_final.columns)}')
print(f'  File size: {os.path.getsize(stock_output_path) / 1024 / 1024:.2f} MB')


Stock data saved
  Records: 38,340
  Columns: ['Date', 'Ticker', 'Log_Return', 'Sector', 'Beta', 'Beta_Group']
  File size: 2.89 MB
